# Quiz 3

Juan Sebastian Alvarez Eraso

Código: 201822427

## Limpieza de datos

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/juanalvarez123/MINE-4101-quiz-3/main/Dataset/insurance.csv', sep = ',')

# Imprimo el dataset
df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [2]:
# Visualizo la cantidad de registros y columnas del dataset
df.shape

(1338, 7)

In [3]:
# Cambio la columna "sex" por valores con 0 y 1
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'male' else 0)

# Cambio la columna "smoker" por valores con 0 y 1
df['smoker'] = df['smoker'].apply(lambda x: 1 if x == 'smoker' else 0)

In [4]:
# Codifico la columna "region"
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Con el parámetro "drop ='first'" se elimina una columna de cada feature y así se evita la multicolinealidad
ct = ColumnTransformer(transformers =[('OneHotEncoder', OneHotEncoder(sparse = False, drop ='first'), ['region'])], remainder ='passthrough')
df2 = ct.fit_transform(df)
df2 = pd.DataFrame(data=df2, columns=ct.get_feature_names_out())

# Imprimo el dataset
df2.head(5)

,OneHotEncoder__region_northwest,OneHotEncoder__region_southeast,OneHotEncoder__region_southwest,remainder__age,remainder__sex,remainder__bmi,remainder__children,remainder__smoker,remainder__charges
0,0.0,0.0,1.0,19.0,0.0,27.900,0.0,0.0,16884.92400
1,0.0,1.0,0.0,18.0,1.0,33.770,1.0,0.0,1725.55230
2,0.0,1.0,0.0,28.0,1.0,33.000,3.0,0.0,4449.46200
3,1.0,0.0,0.0,33.0,1.0,22.705,0.0,0.0,21984.47061
4,1.0,0.0,0.0,32.0,1.0,28.880,0.0,0.0,3866.85520


In [5]:
# Verifico los tipos de datos
df2.dtypes

OneHotEncoder__region_northwest    float64
OneHotEncoder__region_southeast    float64
OneHotEncoder__region_southwest    float64
remainder__age                     float64
remainder__sex                     float64
remainder__bmi                     float64
remainder__children                float64
remainder__smoker                  float64
remainder__charges                 float64
dtype: object

Todos son float64, estamos bien para continuar

## Obtener la variable dependiente y las variables independientes

In [6]:
features = ['OneHotEncoder__region_northwest', 'OneHotEncoder__region_southeast',
            'OneHotEncoder__region_southwest', 'remainder__age', 'remainder__sex',
            'remainder__bmi', 'remainder__children', 'remainder__smoker']
x = df2[features]
y = df2['remainder__charges']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)

print(f'Train: {x_train.shape}')
print(f'Test: {x_test.shape}')

Train: (1070, 8)
Test: (268, 8)


In [7]:
# Se estandariza el dataset usando 'StandarScaler'
name_columns = x_train.columns

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train.values) # Solo para datos de entrenamiento se utiliza "fit_transform"
x_test = sc.transform(x_test.values)

# Los datos se transforman a dataframes para mayor comodidad
x_train = pd.DataFrame(data=x_train, columns=name_columns)
x_test = pd.DataFrame(data=x_test, columns=name_columns)

## Regresión lineal simple

In [8]:
# Se realiza el entrenamiento del modelo
from sklearn.linear_model import LinearRegression

simple_linear_regression = LinearRegression()
simple_linear_regression.fit(x_train, y_train)

LinearRegression()

In [9]:
# Se imprimen los valores de los coeficientes. Los que son más grandes y tienen
# más peso son los que influyen más en la desición de la predicción
print(simple_linear_regression.feature_names_in_)
print(simple_linear_regression.coef_)

['OneHotEncoder__region_northwest' 'OneHotEncoder__region_southeast'
 'OneHotEncoder__region_southwest' 'remainder__age' 'remainder__sex'
 'remainder__bmi' 'remainder__children' 'remainder__smoker']
[-380.01850591 -112.93379397 -565.62983322 3459.018489    586.31183665
 1805.46166939  322.91902786    0.        ]


El valor más pesado es `3459.018489` que corresponde a la columna `age`

In [10]:
# Se predicen los datos
y_pred_train = simple_linear_regression.predict(x_train)
y_pred_test = simple_linear_regression.predict(x_test)

In [11]:
# Encuentro el error cuadrático médio (MSE)
import numpy as np
from sklearn.metrics import mean_squared_error

mse_train_lr = np.sqrt(mean_squared_error(y_train, y_pred_train))
mse_test_lr = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'LinearRegression MSE train: {mse_train_lr}')
print(f'LinearRegression MSE test: {mse_test_lr}')

LinearRegression MSE train: 11330.599663772457
LinearRegression MSE test: 11295.382091920017


### Conclusiones hasta ahora

1. ¿Es aceptable el error obtenido?
Rta/ No, el modelo tiene mucho error. Si lo llevamos a las unidades de costos y cargos en USD se está equivocando por 11330 USD.

2. ¿Hay evidencia de overfitting?
Rta/ No, no hay evidencia de overfitting. Tenemos evidencia de underfitting ya que los 2 errores están muy cerca.

## Gradiente descendente

### SGDRegressor sin función de regularización

In [12]:
# Se realiza el entrenamiento del modelo con penalty=None
from sklearn.linear_model import SGDRegressor

sgd_regressor = SGDRegressor(penalty=None)
sgd_regressor.fit(x_train, y_train)

SGDRegressor(penalty=None)

In [13]:
# Se imprimen los valores de los coeficientes. Los que son más grandes y tienen
# más peso son los que influyen más en la desición de la predicción
print(sgd_regressor.feature_names_in_)
print(sgd_regressor.coef_)

['OneHotEncoder__region_northwest' 'OneHotEncoder__region_southeast'
 'OneHotEncoder__region_southwest' 'remainder__age' 'remainder__sex'
 'remainder__bmi' 'remainder__children' 'remainder__smoker']
[-311.26840092 -289.78796821 -433.80743299 3576.28207706  657.36946356
 1731.74214275  386.85341777    0.        ]


El valor más pesado es `3576.28207706` que corresponde a la columna `age`

In [14]:
# Se predicen los datos
y_pred_train = sgd_regressor.predict(x_train)
y_pred_test = sgd_regressor.predict(x_test)

In [15]:
# Encuentro el error cuadrático médio (MSE)
mse_train_sgd = np.sqrt(mean_squared_error(y_train, y_pred_train))
mse_test_sgd = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'SGDRegressor sin regularización MSE train: {mse_train_sgd}')
print(f'SGDRegressor sin regularización MSE test: {mse_test_sgd}')

SGDRegressor sin regularización MSE train: 11335.444215477337
SGDRegressor sin regularización MSE test: 11326.052588192328


### SGDRegressor con función de regularización L1

In [16]:
# Se realiza el entrenamiento del modelo con penalty=l1
sgd_regressor_l1 = SGDRegressor(penalty='l1')
sgd_regressor_l1.fit(x_train, y_train)

SGDRegressor(penalty='l1')

In [17]:
# Se imprimen los valores de los coeficientes. Los que son más grandes y tienen
# más peso son los que influyen más en la desición de la predicción
print(sgd_regressor_l1.feature_names_in_)
print(sgd_regressor_l1.coef_)

['OneHotEncoder__region_northwest' 'OneHotEncoder__region_southeast'
 'OneHotEncoder__region_southwest' 'remainder__age' 'remainder__sex'
 'remainder__bmi' 'remainder__children' 'remainder__smoker']
[-440.13021201  -48.51278399 -551.85340336 3422.11117213  632.63084645
 1826.76805477  295.94955838    0.        ]


El valor más pesado es `3422.11117213` que corresponde a la columna `age`



In [18]:
# Se predicen los datos
y_pred_train = sgd_regressor_l1.predict(x_train)
y_pred_test = sgd_regressor_l1.predict(x_test)

In [19]:
# Encuentro el error cuadrático médio (MSE)
mse_train_sgd_l1 = np.sqrt(mean_squared_error(y_train, y_pred_train))
mse_test_sgd_l1 = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'SGDRegressor con L1 MSE train: {mse_train_sgd_l1}')
print(f'SGDRegressor con L1 MSE test: {mse_test_sgd_l1}')

SGDRegressor con L1 MSE train: 11331.45787823787
SGDRegressor con L1 MSE test: 11287.974717054609


## Transformación polinomial

### Con 2 grados

In [20]:
# Inicio con 2 grados
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)
x_poly = poly_features.fit_transform(x)

In [21]:
# Obtener variable dependiente y variables independientes
x_train, x_test, y_train, y_test = train_test_split(x_poly, y, test_size=0.20, random_state=1)

print(f'Train: {x_train.shape}')
print(f'Test: {x_test.shape}')

Train: (1070, 44)
Test: (268, 44)


In [22]:
# Se estandariza el dataset usando 'StandarScaler'
sc = StandardScaler()
x_train = sc.fit_transform(x_train) # Solo para datos de entrenamiento se utiliza "fit_transform"
x_test = sc.transform(x_test)

#### Gradiente descendente con función de regularización l1

In [23]:
# Se realiza el entrenamiento del modelo con penalty=l1, alpha=0.1/m
m = 1070
sgd_regressor_l1 = SGDRegressor(penalty='l1', alpha=0.1/m)
sgd_regressor_l1.fit(x_train, y_train)

SGDRegressor(alpha=9.345794392523365e-05, penalty='l1')

In [24]:
# Se predicen los datos
y_pred_train = sgd_regressor_l1.predict(x_train)
y_pred_test = sgd_regressor_l1.predict(x_test)

In [25]:
# Encuentro el error cuadrático médio (MSE)
mse_train_sgd_2g_l1 = np.sqrt(mean_squared_error(y_train, y_pred_train))
mse_test_sgd_2g_l1 = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'SGDRegressor 2 grados, L1 y alpha=0.1/m MSE train: {mse_train_sgd_2g_l1}')
print(f'SGDRegressor 2 grados, L1 y alpha=0.1/m MSE test: {mse_test_sgd_2g_l1}')

SGDRegressor 2 grados, L1 y alpha=0.1/m MSE train: 11262.778734267225
SGDRegressor 2 grados, L1 y alpha=0.1/m MSE test: 11430.175358779212


#### Gradiente descendente con función de regularización l2

In [26]:
# Se realiza el entrenamiento del modelo con penalty=l2, alpha=0.01/m
m = 1070
sgd_regressor_l2 = SGDRegressor(penalty='l2', alpha=0.01/m)
sgd_regressor_l2.fit(x_train, y_train)

SGDRegressor(alpha=9.345794392523365e-06)

In [27]:
# Se predicen los datos
y_pred_train = sgd_regressor_l2.predict(x_train)
y_pred_test = sgd_regressor_l2.predict(x_test)

In [28]:
# Encuentro el error cuadrático médio (MSE)
mse_train_sgd_2g_l2 = np.sqrt(mean_squared_error(y_train, y_pred_train))
mse_test_sgd_2g_l2 = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'SGDRegressor 2 grados, L2 y alpha=0.01/m MSE train: {mse_train_sgd_2g_l2}')
print(f'SGDRegressor 2 grados, L2 y alpha=0.01/m MSE test: {mse_test_sgd_2g_l2}')

SGDRegressor 2 grados, L2 y alpha=0.01/m MSE train: 11281.571541178728
SGDRegressor 2 grados, L2 y alpha=0.01/m MSE test: 11486.53481883472


### Con 5 grados

In [29]:
# Ahora con 5 grados
poly_features = PolynomialFeatures(degree=5, include_bias=False)
x_poly = poly_features.fit_transform(x)

In [30]:
# Obtener variable dependiente y variables independientes
x_train, x_test, y_train, y_test = train_test_split(x_poly, y, test_size=0.20, random_state=1)

print(f'Train: {x_train.shape}')
print(f'Test: {x_test.shape}')

Train: (1070, 1286)
Test: (268, 1286)


In [31]:
# Se estandariza el dataset usando 'StandarScaler'
sc = StandardScaler()
x_train = sc.fit_transform(x_train) # Solo para datos de entrenamiento se utiliza "fit_transform"
x_test = sc.transform(x_test)

#### Gradiente descendente con función de regularización l1

In [32]:
# Se realiza el entrenamiento del modelo con penalty=l1, alpha=0.2/m
m = 1070
sgd_regressor_l1 = SGDRegressor(penalty='l1', alpha=0.2/m)
sgd_regressor_l1.fit(x_train, y_train)

SGDRegressor(alpha=0.0001869158878504673, penalty='l1')

In [33]:
# Se predicen los datos
y_pred_train = sgd_regressor_l1.predict(x_train)
y_pred_test = sgd_regressor_l1.predict(x_test)

In [34]:
# Encuentro el error cuadrático médio (MSE)
mse_train_sgd_5g_l1 = np.sqrt(mean_squared_error(y_train, y_pred_train))
mse_test_sgd_5g_l1 = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'SGDRegressor 5 grados, L1 y alpha=0.2/m MSE train: {mse_train_sgd_5g_l1}')
print(f'SGDRegressor 5 grados, L1 y alpha=0.2/m MSE test: {mse_test_sgd_5g_l1}')

SGDRegressor 5 grados, L1 y alpha=0.2/m MSE train: 64419123708.09778
SGDRegressor 5 grados, L1 y alpha=0.2/m MSE test: 48211477480.66672


#### Gradiente descendente con función de regularización l2

In [35]:
# Se realiza el entrenamiento del modelo con penalty=l2, alpha=0.02/m
m = 1070
sgd_regressor_l2 = SGDRegressor(penalty='l2', alpha=0.02/m)
sgd_regressor_l2.fit(x_train, y_train)

SGDRegressor(alpha=1.869158878504673e-05)

In [36]:
# Se predicen los datos
y_pred_train = sgd_regressor_l2.predict(x_train)
y_pred_test = sgd_regressor_l2.predict(x_test)

In [37]:
# Encuentro el error cuadrático médio (MSE)
mse_train_sgd_5g_l2 = np.sqrt(mean_squared_error(y_train, y_pred_train))
mse_test_sgd_5g_l2 = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'SGDRegressor 5 grados, L2 y alpha=0.02/m MSE train: {mse_train_sgd_5g_l2}')
print(f'SGDRegressor 5 grados, L2 y alpha=0.02/m MSE test: {mse_test_sgd_5g_l2}')

SGDRegressor 5 grados, L2 y alpha=0.02/m MSE train: 23971499653.601357
SGDRegressor 5 grados, L2 y alpha=0.02/m MSE test: 23299439048.57683


## Resumen de MSE obtenidos

In [38]:
print(f'LinearRegression MSE train: {mse_train_lr}')
print(f'LinearRegression MSE test: {mse_test_lr}')
print('')
print(f'SGDRegressor sin regularización MSE train: {mse_train_sgd}')
print(f'SGDRegressor sin regularización MSE test: {mse_test_sgd}')
print('')
print(f'SGDRegressor con L1 MSE train: {mse_train_sgd_l1}')
print(f'SGDRegressor con L1 MSE test: {mse_test_sgd_l1}')
print('')
print(f'SGDRegressor 2 grados, L1 y alpha=0.1/m MSE train: {mse_train_sgd_2g_l1}')
print(f'SGDRegressor 2 grados, L1 y alpha=0.1/m MSE test: {mse_test_sgd_2g_l1}')
print('')
print(f'SGDRegressor 2 grados, L2 y alpha=0.01/m MSE train: {mse_train_sgd_2g_l2}')
print(f'SGDRegressor 2 grados, L2 y alpha=0.01/m MSE test: {mse_test_sgd_2g_l2}')
print('')
print(f'SGDRegressor 5 grados, L1 y alpha=0.2/m MSE train: {mse_train_sgd_5g_l1}')
print(f'SGDRegressor 5 grados, L1 y alpha=0.2/m  MSE test: {mse_test_sgd_5g_l1}')
print('')
print(f'SGDRegressor 5 grados, L2 y alpha=0.02/m MSE train: {mse_train_sgd_5g_l2}')
print(f'SGDRegressor 5 grados, L2 y alpha=0.02/m MSE test: {mse_test_sgd_5g_l2}')

LinearRegression MSE train: 11330.599663772457
LinearRegression MSE test: 11295.382091920017

SGDRegressor sin regularización MSE train: 11335.444215477337
SGDRegressor sin regularización MSE test: 11326.052588192328

SGDRegressor con L1 MSE train: 11331.45787823787
SGDRegressor con L1 MSE test: 11287.974717054609

SGDRegressor 2 grados, L1 y alpha=0.1/m MSE train: 11262.778734267225
SGDRegressor 2 grados, L1 y alpha=0.1/m MSE test: 11430.175358779212

SGDRegressor 2 grados, L2 y alpha=0.01/m MSE train: 11281.571541178728
SGDRegressor 2 grados, L2 y alpha=0.01/m MSE test: 11486.53481883472

SGDRegressor 5 grados, L1 y alpha=0.2/m MSE train: 64419123708.09778
SGDRegressor 5 grados, L1 y alpha=0.2/m  MSE test: 48211477480.66672

SGDRegressor 5 grados, L2 y alpha=0.02/m MSE train: 23971499653.601357
SGDRegressor 5 grados, L2 y alpha=0.02/m MSE test: 23299439048.57683


## Conclusiones

1. Si fue posible mejorar el error encontrado en el primer modelo que fue con LinearRegression (MSE train: 11330.599663772457), el modelo que lo mejoró fue SGDRegressor con 2 grados en la transformación polinomial, regularización L1 y alpha=0.1/m (MSE train: 11262.778734267225), sin embargo la diferencia no fue mucha.
2. Los hiper parámetros del modelo con el menor error son 2 grados en la transformación polinomial, regularización L1 y alpha=0.1/m.
3. Los atributos que parecen ser más importantes son aplicar la regularización y utilizar pocos grados en la transformación polinomial. Con muchos grados en la transformación polinomial el error sube demasiado.